In [47]:
import pandas as pd

What we're aiming for is something like this

```json
{
	"E14999999":{
		"name": "Fake news",
		"code": "E14999999",
		"description": "This constituency is great!",
		"headlines": [{"value":1,"name":"People"},{"value":1,"name":"People"},{"value":1,"name":"People"},{"value":1,"name":"People"}],
		"similar": { "People": 4 },
		"leastsimilar": { "People": 1000 },
		"neet": {
			"something": [34,56,24,19,25],
			"another": {}
		},
		"employment": {
			
		}
	}
}
```

In [48]:
descriptions=pd.read_csv('../../data/reference/constituency-descriptions.csv', index_col='PCON22CD').fillna('')

In [49]:
pcons=pd.read_json('../../src/_data/areas/reference/pcon.json')

In [50]:
summary=pd.merge(pcons, descriptions, how='left', left_on='PCON22CD', right_index=True).set_index('PCON22CD')

summary.rename(columns={ 'PCON22NM': 'name'}, inplace=True)
summary.rename(columns=lambda x: x.lower(), inplace=True)

In [51]:
summary.to_json('../../src/generated/areas/constituency/_data/summary.json', orient='index')